# WIP: Scan in $(hkl)$ coordinates

This notebook demonstrates how to scan in $(hkl)$ coordinates. It uses the
simulated 4-circle geometry from the `"hkl_soleil"` solver. The wavelength and
sample are whatever the simulator provides as defaults.

**Important**:  It is possible to scan in any combination of reciprocal axes or to
scan in any combination of real-space axes.  You are not allowed to scan in a
mix of reciprocal and real-space axes.

## Setup

First, create the simulated 4-circle diffractometer object (`e4cv`).

In [1]:
from hklpy2 import SimulatedE4CV

e4cv = SimulatedE4CV(name="e4cv")

Setup Bluesky for running the scans with the `RE` object.  The `bec` object will
show a table of the data collected for each scan.  

For this simple demonstration, we won't add a databroker catalog.

In [2]:
from bluesky import RunEngine, plans as bp
from bluesky.callbacks.best_effort import BestEffortCallback

bec = BestEffortCallback()
RE = RunEngine()
RE.subscribe(bec)
bec.disable_plots()

We'll **import a simulator** (ready to use) from the `ophyd` package as a noisy detector.

In [3]:
from ophyd.sim import noisy_det

## (h10) scan

Scan the (reciprocal space) $h$ axis from -0.5 to +0.5 with $k=1$ and $l=0$.
This is called an $(h10)$ scan.

<details>

The computation to convert reciprocal-space values $(h,k,l)$ into real-space
angles ($\omega$, $\chi$, $\phi$, $2\theta$) is called the `forward()`
transformation.  The transformation is not necessarily unique.  The most common
way to reduce the number of *solutions* is to tell the solver which `mode` to
use.  The `mode` adds an additional pre-designed rule that constrains the
acceptable solutions.  The solver's geometry (in this case `E4CV`) provides the
list of known modes.

Note: Even with a chosen mode, the solution might not be unique.  In such cases,
the first solution returned by the `forward()` transformation is chosen.  The
user can change this by providing a different function for the diffractometer's
`_forward_solution` attribute.  The default is the
`hklpy2.diffract.pick_first_item()` function.

</details>

Here, the diffractometer starts with `"bissector"` mode (requires `tth = 2*omega`).

In [4]:
print(f"{e4cv.operator.solver.mode=!r}")
e4cv.k.move(1)
e4cv.l.move(0)
RE(bp.scan([noisy_det], e4cv.h, -0.5, 0.5, 11))

e4cv.operator.solver.mode='bissector'


Transient Scan ID: 1     Time: 2024-07-29 10:45:44
Persistent Unique Scan ID: 'a66f79a6-96cb-441d-a5aa-71e2328fc68c'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |     e4cv_h |  noisy_det |
+-----------+------------+------------+------------+
|         1 | 10:45:44.2 |     -0.500 |      0.996 |
|         2 | 10:45:44.2 |     -0.400 |      0.985 |
|         3 | 10:45:44.2 |     -0.300 |      1.049 |
|         4 | 10:45:44.2 |     -0.200 |      0.980 |
|         5 | 10:45:44.2 |     -0.100 |      1.032 |
|         6 | 10:45:44.2 |      0.000 |      0.966 |
|         7 | 10:45:44.2 |      0.100 |      0.969 |
|         8 | 10:45:44.2 |      0.200 |      1.014 |
|         9 | 10:45:44.3 |      0.300 |      0.911 |
|        10 | 10:45:44.3 |      0.400 |      0.952 |
|        11 | 10:45:44.3 |      0.500 |      1.020 |
+-----------+------------+------------+------------+
generator scan ['a66f79a6'

('a66f79a6-96cb-441d-a5aa-71e2328fc68c',)

**Clearly we see** that $h$ has been stepped across the range of -0.5 to +0.5.
Values for the noisy detector have been reported at each step.  But we want to
know about *all* the $hkl$ and angle values so we can observe the effects of
`"bissector"` mode.

### Scan again, showing all $(hkl)$ and real-space axes

Repeat the scan, same as before with a slight variation.  This time, add the
`e4cv` object as an additional detector.

In [5]:
print(f"{e4cv.operator.solver.mode=!r}")
e4cv.k.move(1)
e4cv.l.move(0)
RE(bp.scan([noisy_det, e4cv], e4cv.h, -0.5, 0.5, 11))

e4cv.operator.solver.mode='bissector'


Transient Scan ID: 2     Time: 2024-07-29 10:45:44
Persistent Unique Scan ID: '78c80bd6-ebed-45c5-866d-14f05769844f'
New stream: 'primary'
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|   seq_num |       time |     e4cv_h |  noisy_det |     e4cv_k |     e4cv_l | e4cv_omega |   e4cv_chi |   e4cv_phi |   e4cv_tth |
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|         1 | 10:45:44.4 |     -0.500 |      0.960 |      1.000 |      0.000 |    -33.988 |    -63.435 |     90.000 |    -67.976 |
|         2 | 10:45:44.4 |     -0.400 |      0.914 |      1.000 |      0.000 |    -32.583 |    -68.199 |     90.000 |    -65.165 |
|         3 | 10:45:44.4 |     -0.300 |      0.950 |      1.000 |     -0.000 |    -31.468 |    -73.301 |     90.000 |    -62.935 |
|         4 | 10:45:44.4 |     -0.2

('78c80bd6-ebed-45c5-866d-14f05769844f',)

## What other modes are available?

In [6]:
e4cv.operator.solver.modes

['bissector',
 'constant_omega',
 'constant_chi',
 'constant_phi',
 'double_diffraction',
 'psi_constant']

## Scan $(h10)$ holding $\omega$ at -30 degrees

Set the mode to `"constant_omega"`, then set $\omega=-30$ degrees.

In [7]:
e4cv.operator.solver.mode = "constant_omega"
e4cv.omega.move(-30)
print(f"{e4cv.omega.position=!r}")

e4cv.omega.position=-30


**Run the scan again** with the same command.

In [8]:
print(f"{e4cv.operator.solver.mode=!r}")
e4cv.k.move(1)
e4cv.l.move(0)
RE(bp.scan([noisy_det, e4cv], e4cv.h, -0.5, 0.5, 11))

e4cv.operator.solver.mode='constant_omega'


Transient Scan ID: 3     Time: 2024-07-29 10:45:44
Persistent Unique Scan ID: 'fc29120e-ea36-4960-b78e-580938dd85ff'
New stream: 'primary'
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|   seq_num |       time |     e4cv_h |  noisy_det |     e4cv_k |     e4cv_l | e4cv_omega |   e4cv_chi |   e4cv_phi |   e4cv_tth |
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|         1 | 10:45:44.7 |     -0.500 |      1.054 |      1.000 |      0.000 |    -30.000 |    -63.714 |     81.054 |    -67.976 |
|         2 | 10:45:44.7 |     -0.400 |      0.939 |      1.000 |     -0.000 |    -30.000 |    -68.345 |     83.031 |    -65.165 |
|         3 | 10:45:44.7 |     -0.300 |      1.060 |      1.000 |     -0.000 |    -30.000 |    -73.364 |     84.887 |    -62.935 |
|         4 | 10:45:44.7 |    

('fc29120e-ea36-4960-b78e-580938dd85ff',)

## Scan $(\bar{1}kl)$ holding $\omega$ at -30 degrees

Keep mode as `"constant_omega"` and $\omega=-30$.  Set $h=-1$ and scan $k$ & $l$.

In [9]:
e4cv.h.move(-1)
print(f"{e4cv.operator.solver.mode=!r}")
RE(bp.scan([noisy_det, e4cv], e4cv.k, 0.9, 1.1, e4cv.l, -0.6, -0.4, 11))

e4cv.operator.solver.mode='constant_omega'


Transient Scan ID: 4     Time: 2024-07-29 10:45:44
Persistent Unique Scan ID: '48c0cf76-361d-435a-935a-ae45562a5c8e'
New stream: 'primary'
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|   seq_num |       time |     e4cv_k |     e4cv_l |  noisy_det |     e4cv_h | e4cv_omega |   e4cv_chi |   e4cv_phi |   e4cv_tth |
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|         1 | 10:45:45.0 |      0.900 |     -0.600 |      1.000 |     -1.000 |    -30.000 |    -39.821 |     36.793 |    -94.876 |
|         2 | 10:45:45.0 |      0.920 |     -0.580 |      1.038 |     -1.000 |    -30.000 |    -40.796 |     37.124 |    -95.244 |
|         3 | 10:45:45.0 |      0.940 |     -0.560 |      1.091 |     -1.000 |    -30.000 |    -41.770 |     37.424 |    -95.659 |
|         4 | 10:45:45.0 |    

('48c0cf76-361d-435a-935a-ae45562a5c8e',)

## Scan $(h10)$ holding $\psi$ at 25 degrees around $(100)$

Set the mode to `"psi_constant"`, then set $h_2=1, k_2=0, l_2=0$ & $\psi=25$ degrees.

TODO: What is $\psi$?  What is $(h_2, k_2, l_2)$?  Is enabled by solver yet?

In [10]:
e4cv.operator.solver.mode = "psi_constant"

# TODO: Can this be even easier?
extras = {
    "h2": 1,
    "k2": 0,
    "l2": 0,
    "psi": 25,
}
e4cv.operator.solver.extras = extras

In [11]:
print(f"{e4cv.operator.solver.mode=!r}")
print(f"{e4cv.operator.solver.extras=!r}")
RE(bp.scan([noisy_det, e4cv], e4cv.k, 0.9, 1.1, e4cv.l, -0.6, -0.4, 11))

e4cv.operator.solver.mode='psi_constant'
e4cv.operator.solver.extras={'h2': 1.0, 'k2': 0.0, 'l2': 0.0, 'psi': 25.0}


Transient Scan ID: 5     Time: 2024-07-29 10:45:45
Persistent Unique Scan ID: '320521d0-e29a-4067-b9d7-dfae2cf0fd77'
New stream: 'primary'
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|   seq_num |       time |     e4cv_k |     e4cv_l |  noisy_det |     e4cv_h | e4cv_omega |   e4cv_chi |   e4cv_phi |   e4cv_tth |
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|         1 | 10:45:45.3 |      0.900 |     -0.600 |      1.036 |     -1.000 |      3.258 |    -74.691 |    -18.768 |    -94.876 |
|         2 | 10:45:45.3 |      0.920 |     -0.580 |      1.018 |     -1.000 |      2.461 |    -76.033 |    -18.698 |    -95.244 |
|         3 | 10:45:45.3 |      0.940 |     -0.560 |      0.971 |     -1.000 |      1.63

('320521d0-e29a-4067-b9d7-dfae2cf0fd77',)

## Scan $\psi$ around $(100)$ with sample oriented at $(101)$

Set the mode to `"psi_constant"`, then set $h_2=1, k_2=0, l_2=0$. Step scan
$\psi$ through desired range, setting before reporting at each step.


In [12]:
# from lib_demo_scan import scan_extra_parameter
%run -im lib_demo_scan
e4cv.wh(full=True)

diffractometer='e4cv'
Sample(name='sample', lattice=Lattice(a=1, system='cubic'))
HklSolver(name='hkl_soleil', version='5.0.0.3511', geometry='E4CV', engine_name='hkl', mode='psi_constant')
wavelength=1.0
h=-1.0 k=1.1 l=-0.4
omega=-6.0135 chi=-87.0346 phi=-18.7687 tth=-100.6611
h2=1.0 k2=0 l2=0 psi=25.0


In [13]:
print(f"{e4cv.operator.solver.mode=!r}")
RE(
    scan_extra_parameter(
        e4cv,
        detectors=[noisy_det],
        pseudos=(1, 0, 1),
        axis="psi",
        start=4,
        finish=44,
        num=11,
        extras=dict(h2=1, k2=0, l2=0),
    ),
    collector.receiver
)

e4cv.operator.solver.mode='psi_constant'


Transient Scan ID: 6     Time: 2024-07-29 10:45:45
Persistent Unique Scan ID: '4207283c-fc7a-4767-8984-54cbb82ed28a'
New stream: 'primary'
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|   seq_num |       time |  noisy_det |     e4cv_h |     e4cv_k |     e4cv_l | e4cv_omega |   e4cv_chi |   e4cv_phi |   e4cv_tth |        psi |
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|         1 | 10:45:45.6 |      0.991 |      1.000 |     -0.000 |      1.000 |   -135.000 |   -176.000 |    -45.000 |    -90.000 |      4.000 |
|         2 | 10:45:45.6 |      0.991 |      1.000 |      0.000 |      1.000 |   -135.000 |   -172.000 |    -45.000 |    -90.000 |      8.000 |
|         3 | 10:45:45.6 |      0.991 |      1.000 |     -0.000 |      1.000 |   -135.000 |   -168

('4207283c-fc7a-4767-8984-54cbb82ed28a',)

Show all the documents published from this run.

In [14]:
collector.documents

[('start',
  {'uid': '4207283c-fc7a-4767-8984-54cbb82ed28a',
   'time': 1722267945.581714,
   'versions': {'ophyd': '1.9.0', 'bluesky': '1.13.0a3'},
   'scan_id': 6,
   'plan_type': 'generator',
   'plan_name': 'scan_extra_parameter',
   'diffractometer': {'name': 'e4cv',
    'solver': 'hkl_soleil',
    'geometry': 'E4CV',
    'engine': 'hkl',
    'mode': 'psi_constant',
    'extra_axes': ['h2', 'k2', 'l2', 'psi']},
   'axis': 'psi',
   'start': 4,
   'finish': 44,
   'num': 11,
   'pseudos': (1, 0, 1),
   'reals': None,
   'extras': {'h2': 1, 'k2': 0, 'l2': 0},
   'transformation': 'forward'}),
 ('descriptor',
  {'configuration': {'noisy_det': {'data': {'noisy_det_Imax': 1,
      'noisy_det_center': 0,
      'noisy_det_sigma': 1,
      'noisy_det_noise': 'uniform',
      'noisy_det_noise_multiplier': 0.1},
     'timestamps': {'noisy_det_Imax': 1722267944.1981087,
      'noisy_det_center': 1722267944.1981025,
      'noisy_det_sigma': 1722267944.1981163,
      'noisy_det_noise': 1722267